# CNN Model 1

## The approach:

### Without time feature
### Pass 1x29 vectors into a convolutional layer, with kernel size 29, with some D number of filters
### Add extra conv and dense layer to the model to see the effect

## CNN1: single conv layer single dense layer:

#### Imports, data set up 

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import numpy
from numpy.random import seed
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import keras

data = pd.read_csv("creditcard.csv")

# Normalise and reshape the Amount column, so it's values lie between -1 and 1
from sklearn.preprocessing import StandardScaler
data['norm_Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))

# Drop the old Amount column and also the Time column as we don't want to include this at this stage
data = data.drop(['Time', 'Amount'], axis=1)

# Assign variables x and y corresponding to row data and it's class value
X = data.loc[:, data.columns != 'Class']
y = data.loc[:, data.columns == 'Class']

#### Function to create and return CNN1 model

In [4]:
# Function to create model
def create_model():
    # create model
    seed(2017)
    conv = Sequential()
    conv.add(Conv1D(256, 29, input_shape=(29, 1), activation='relu'))
    conv.add(Flatten())
    conv.add(Dense(300, activation = 'relu'))
    conv.add(Dense(2, activation = 'softmax'))

    sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
    
    # Compile model
    conv.compile(loss='categorical_crossentropy', optimizer=sgd)
    return conv
 

In [5]:
# Setting up dataframe table properties
log_cols=["Name", "F1 Score", "Precision", "Recall", "Training Time"]
log = pd.DataFrame(columns=log_cols)

#### Function to smote data, from before

In [6]:
def smote_data(x_data, y_data):
    from collections import Counter
    from imblearn.over_sampling import SMOTE
    sm = SMOTE()
    X_res, y_res = sm.fit_sample(x_data, y_data)
    print('Resampling the data with SMOTE. . .')
    print('Resampled training dataset shape {}'.format(Counter(y_res)))

    return X_res, y_res

#### This custom cross val function follows the same method as before but with a functional parameter create_model to allow passing various CNN model creating functions and hence reuse this code

In [7]:
def custom_cross_val(X, y, create_model, n):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.base import clone
    import datetime
    from sklearn.metrics import precision_recall_fscore_support
    
    print 'Cross validating... \n'
    skfolds = StratifiedKFold(n_splits=n, random_state=42)
    
    precision = []
    recall = []
    f1score = []
    elapsed_times = []
    cv = 0
    
    for train_index, test_index in skfolds.split(X, y):
        cv=cv+1
        print len(train_index)
        clone_clf = create_model()
        X_train_folds = X.iloc[train_index]
        y_train_folds = y.iloc[train_index]
        X_test_fold = X.iloc[test_index]
        y_test_fold = y.iloc[test_index]
        
        print len(y_train_folds[y_train_folds['Class']==1])
        X_res, y_res = smote_data(X_train_folds, y_train_folds )
        
        print X_res.shape, type(X_res)
        print y_res.shape

        X_train = X_res.reshape(X_res.shape[0], 29, 1)
        Y_train = y_res.reshape(y_res.shape[0], 1)
        X_test = X_test_fold.values.reshape(X_test_fold.values.shape[0], 29, 1)
        Y_test = y_test_fold.values.reshape(y_test_fold.values.shape[0], 1)

        Y_test = keras.utils.to_categorical(Y_test)
        Y_train = keras.utils.to_categorical(Y_train)
        print Y_test.shape
        print Y_train.shape
        
        
        start = datetime.datetime.now()
        
        print('Fitting the model... CV[{}]'.format(cv)) 
        
        clone_clf.fit(X_train, Y_train, batch_size = 500, epochs = 25, verbose =1)
        end = datetime.datetime.now()
        elapsed = end - start
        elapsed_times.append(elapsed)
        
        y_pred = clone_clf.predict(X_test)
        
        # Set cut off point for class boundaries
        cutt_off_tr = 0.5
        y_pred[np.where(y_pred>=cutt_off_tr)] = 1
        y_pred[np.where(y_pred<cutt_off_tr)]  = 0
        
        prfs = precision_recall_fscore_support(Y_test, y_pred, labels=[0])
        
        precision.append(prfs[0][1])
        recall.append(prfs[1][1])
        f1score.append(prfs[2][1])  
    
    average_timedelta = sum(elapsed_times, datetime.timedelta(0)) / len(elapsed_times)
    entry = ['CNN Model 1', np.mean(f1score), np.mean(precision), np.mean(recall), average_timedelta]
    print('Mean scores: ', entry )
    return entry

In [8]:
results = custom_cross_val(X, y, create_model, 3)

log_entry = pd.DataFrame([results], columns=log_cols)
log = log.append(log_entry)

# Replace table index by the Classifier column
log.set_index('Name', inplace=True)


Cross validating... 

189871
328

/Users/HarryG1/Documents/University/Year_3/DeepLearning-CreditCardFraud/devEnv/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Resampling the data with SMOTE. . .
Resampled training dataset shape Counter({0: 189543, 1: 189543})
(379086, 29) <type 'numpy.ndarray'>
(379086,)
(94936, 2)
(379086, 2)
Fitting the model... CV[1]
Epoch 1/25
379086/379086 [==============================] - 5s 12us/step - loss: 0.0316
Epoch 2/25
379086/379086 [==============================] - 4s 12us/step - loss: 0.0058
Epoch 3/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0038
Epoch 4/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0030
Epoch 5/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0029
Epoch 6/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0079
Epoch 7/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0031
Epoch 8/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0025
Epoch 9/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0022
Epoch 10/2

## CNN1.2: Added Conv and Dense layer

In [9]:
# Function to create model, required for KerasClassifier
def create_model_2():
    # create model
    seed(2017)
    conv = Sequential()
    conv.add(Conv1D(256, 29, input_shape=(29, 1), activation='relu'))
    conv.add(Conv1D(256, 1, activation='relu'))
    conv.add(Flatten())

    conv.add(Dense(300, activation = 'relu'))
    conv.add(Dense(100, activation = 'relu'))
    conv.add(Dense(2, activation = 'softmax'))

    sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
    
    # Compile model
    conv.compile(loss='categorical_crossentropy', optimizer=sgd)
    return conv
 

In [10]:
results1_2 = custom_cross_val(X, y, create_model_2, 3)
results1_2[0] = 'CNN Model 1.2'


Cross validating... 

189871
328
Resampling the data with SMOTE. . .
Resampled training dataset shape Counter({0: 189543, 1: 189543})
(379086, 29) <type 'numpy.ndarray'>
(379086,)
(94936, 2)
(379086, 2)
Fitting the model... CV[1]
Epoch 1/25
379086/379086 [==============================] - 10s 25us/step - loss: 0.0288
Epoch 2/25
379086/379086 [==============================] - 9s 23us/step - loss: 0.0052
Epoch 3/25
379086/379086 [==============================] - 9s 23us/step - loss: 0.0038
Epoch 4/25
379086/379086 [==============================] - 9s 23us/step - loss: 0.0029
Epoch 5/25
379086/379086 [==============================] - 9s 24us/step - loss: 0.0025
Epoch 6/25
379086/379086 [==============================] - 9s 24us/step - loss: 0.0026
Epoch 7/25
379086/379086 [==============================] - 9s 23us/step - loss: 0.0021
Epoch 8/25
379086/379086 [==============================] - 9s 23us/step - loss: 0.0025
Epoch 9/25
379086/379086 [==============================] - 9s 23

In [27]:
log_entry = pd.DataFrame([results], columns=log_cols)
log = log.append(log_entry)
log_entry = pd.DataFrame([results1_2], columns=log_cols)
log = log.append(log_entry)

In [28]:
log

,F1 Score,Name,Precision,Recall,Training Time
0,0.696680,CNN Model 1,0.628969,0.821138,00:01:42.882115
0,0.745591,CNN Model 1.2,0.698229,0.817073,00:03:42.131883


In [29]:
# Replace table index by the Classifier column
log.set_index('Name', inplace=True)

In [30]:
log

,F1 Score,Precision,Recall,Training Time
Name,,,,
CNN Model 1,0.696680,0.628969,0.821138,00:01:42.882115
CNN Model 1.2,0.745591,0.698229,0.817073,00:03:42.131883


## Results

In [31]:
print log

               F1 Score  Precision    Recall   Training Time
Name                                                        
CNN Model 1    0.696680   0.628969  0.821138 00:01:42.882115
CNN Model 1.2  0.745591   0.698229  0.817073 00:03:42.131883


In [38]:
import pickle
pickle.dump( log , open( "cnnv1_results.p", "wb" ) )

## Model Summaries

In [32]:
conv1 = create_model()
conv1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 1, 256)            7680      
_________________________________________________________________
flatten_7 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 300)               77100     
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 602       
Total params: 85,382
Trainable params: 85,382
Non-trainable params: 0
_________________________________________________________________


In [33]:
conv2 = create_model_2()
conv2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_11 (Conv1D)           (None, 1, 256)            7680      
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 1, 256)            65792     
_________________________________________________________________
flatten_8 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 300)               77100     
_________________________________________________________________
dense_19 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 202       
Total params: 180,874
Trainable params: 180,874
Non-trainable params: 0
_________________________________________________________________


## Evaluation

We see that in this simple CNN approach, we do not meet the best we have seen amongst baseline models. However, looking a bit more specifically, we see that Recall scores are comparable, but without the drastic low in precision scores, like we had in some baseline models. 

Comparing specifically with the (tuned) Random Forest classifier:
We see that the Recall is approximately the same, it is the precision that is lower.

Comparing with the Smote results across all baseline models we can see that our CNN models are higher in F1-score except the RF classifier. In particular the precision scores are comparable to the second best performing classifier the MLP. 

In going from CNN1 to CNN1.2, we see that the major performance increase comes from raising the precision score, resulting in a higher F1 overall. This means that our approach of adding an extra conv layer and dense layer helped to improve precision of the model. The training time however, obviously took longer, almost double the time.

## Same as before, but trying higher class threshold

In [36]:
def custom_cross_val_2(X, y, create_model, n):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.base import clone
    import datetime
    from sklearn.metrics import precision_recall_fscore_support
    
    print 'Cross validating... \n'
    skfolds = StratifiedKFold(n_splits=n, random_state=42)
    
    precision = []
    recall = []
    f1score = []
    elapsed_times = []
    cv = 0
    
    for train_index, test_index in skfolds.split(X, y):
        cv=cv+1
        print len(train_index)
        clone_clf = create_model()
        X_train_folds = X.iloc[train_index]
        y_train_folds = y.iloc[train_index]
        X_test_fold = X.iloc[test_index]
        y_test_fold = y.iloc[test_index]
        
        print len(y_train_folds[y_train_folds['Class']==1])
        X_res, y_res = smote_data(X_train_folds, y_train_folds )
        
        print X_res.shape, type(X_res)
        print y_res.shape

        X_train = X_res.reshape(X_res.shape[0], 29, 1)
        Y_train = y_res.reshape(y_res.shape[0], 1)
        X_test = X_test_fold.values.reshape(X_test_fold.values.shape[0], 29, 1)
        Y_test = y_test_fold.values.reshape(y_test_fold.values.shape[0], 1)

        Y_test = keras.utils.to_categorical(Y_test)
        Y_train = keras.utils.to_categorical(Y_train)
        print Y_test.shape
        print Y_train.shape
        
        
        start = datetime.datetime.now()
        
        print('Fitting the model... CV[{}]'.format(cv)) 
        
        clone_clf.fit(X_train, Y_train, batch_size = 500, epochs = 25, verbose =1)
        end = datetime.datetime.now()
        elapsed = end - start
        elapsed_times.append(elapsed)
        
        y_pred = clone_clf.predict(X_test)
        
        # Set cut off point for class boundaries
        cutt_off_tr = 0.6
        y_pred[np.where(y_pred>=cutt_off_tr)] = 1
        y_pred[np.where(y_pred<cutt_off_tr)]  = 0
        
        prfs = precision_recall_fscore_support(Y_test, y_pred, labels=[0])
        
        precision.append(prfs[0][1])
        recall.append(prfs[1][1])
        f1score.append(prfs[2][1])  
    
    average_timedelta = sum(elapsed_times, datetime.timedelta(0)) / len(elapsed_times)
    entry = ['CNN Model 1.1', np.mean(f1score), np.mean(precision), np.mean(recall), average_timedelta]
    print('Mean scores: ', entry )
    return entry

In [37]:
results2 = custom_cross_val(X, y, create_model, 3)

Cross validating... 

189871
328
Resampling the data with SMOTE. . .
Resampled training dataset shape Counter({0: 189543, 1: 189543})
(379086, 29) <type 'numpy.ndarray'>
(379086,)
(94936, 2)
(379086, 2)
Fitting the model... CV[1]
Epoch 1/25
379086/379086 [==============================] - 5s 12us/step - loss: 0.0315
Epoch 2/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0057
Epoch 3/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0037
Epoch 4/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0038
Epoch 5/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0028
Epoch 6/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0023
Epoch 7/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0023
Epoch 8/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0024
Epoch 9/25
379086/379086 [==============================] - 4s 11u